clustering con predict + functions

In [1]:
import sys
sys.path.append('../../')

In [2]:
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt

from modules import machine_learning_utils as mlu

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler

imp_mean = IterativeImputer(random_state=0)
imp_mean2 = SimpleImputer(strategy='constant', fill_value='missing',verbose=0,add_indicator=True)
scaler = MinMaxScaler()

from kmodes.kprototypes import KPrototypes
from sklearn.metrics import silhouette_score

import gower
import heapq

pd.set_option("display.max_rows",None)
pd.set_option("display.max_columns",None)

# 1.0 Data retrieval

In [3]:
df = pd.read_pickle("../../data_lake/output/train.pkl")

# 2.0 Features Engineering

In [4]:
df = mlu.features_eng(df,'clustering')

In [5]:
cols = ['card1', 'card2', 'card3', 'card5', 'M4', 'id_21', 'id_22', 'id_23', 'id_24', 'id_25',
       'id_26', 'id_27', 'id_28', 'id_29', 'id_30', 'id_32', 'id_33', 'id_34',
       'id_35', 'id_36', 'id_37', 'id_38','num_transaction_per_time','addr1','P_emaildomain','R_emaildomain']

In [6]:
df = df.drop(columns=cols)

In [7]:
df = df[df['isFraud']==0]
df = df.drop(columns='isFraud')

In [8]:
df = df.drop(columns=['addr2','multi_transaction_per_time'])

In [9]:
df.sample(1)

,TransactionAmt,ProductCD,card4,card6,DeviceType,max_c,max_d,customer_id,browser_enc,device_info_v4
110990,20.0,S,visa,debit,mobile,2.0,0.666666,1527,chrome,lg


# 3.0 Training clustering

In [10]:
def clustering_preparation(df):
    """
    
    """
    numerical_cols = df.select_dtypes(include=['int64', 'float64']).columns
    categorical_cols = df.select_dtypes(include=['object', 'bool']).columns
    for i in df:
        if i in numerical_cols:
            if i != 'customer_id':
                df[i] = imp_mean.fit_transform(X = df[i].values.reshape(-1,1))
                df[i] = scaler.fit_transform(X = df[i].values.reshape(-1,1))
    for i in df:
        if i in categorical_cols:
            df[i] = df[i].astype(str)
            df[i] = imp_mean2.fit_transform(X = df[i].values.reshape(-1,1))
    return df, imp_mean, imp_mean2, scaler

In [11]:
df, imp_mean, imp_mean2, scaler = clustering_preparation(df)

In [12]:
df.sample(3)

,TransactionAmt,ProductCD,card4,card6,DeviceType,max_c,max_d,customer_id,browser_enc,device_info_v4
76279,0.018073,C,visa,debit,mobile,0.000000,0.158099,16,chrome,hisense
109512,0.007796,C,visa,debit,desktop,0.008359,0.062142,72,chrome,other
26811,0.007767,C,mastercard,debit,desktop,0.000356,0.009954,430,safari,other


In [13]:
def clustering_encoding(df):
    """
    
    """
    # Product
    df['product_enc'] = 999
    df['product_enc'] = df['product_enc'].mask(df['ProductCD']=='C',0)
    df['product_enc'] = df['product_enc'].mask(df['ProductCD']=='H',1)
    df['product_enc'] = df['product_enc'].mask(df['ProductCD']=='S',2)
    df['product_enc'] = df['product_enc'].mask(df['ProductCD']=='R',3)
    df = df.drop(columns='ProductCD')
    # Card4
    df['card4_enc'] = 999
    df['card4_enc'] = df['card4_enc'].mask(df['card4']=='visa',0)
    df['card4_enc'] = df['card4_enc'].mask(df['card4']=='mastercard',1)
    df['card4_enc'] = df['card4_enc'].mask(df['card4']=='american express',2)
    df['card4_enc'] = df['card4_enc'].mask(df['card4']=='discover',3)
    df = df.drop(columns='card4')
    # Card6
    df['card6_enc'] = 999
    df['card6_enc'] = df['card6_enc'].mask(df['card6']=='debit',0)
    df['card6_enc'] = df['card6_enc'].mask(df['card6']=='credit',1)
    df = df.drop(columns='card6')
    # Device Type
    df['DeviceType_enc'] = 999
    df['DeviceType_enc'] = df['DeviceType_enc'].mask(df['DeviceType']=='mobile',0)
    df['DeviceType_enc'] = df['DeviceType_enc'].mask(df['DeviceType']=='desktop',1)
    df['DeviceType_enc'] = df['DeviceType_enc'].mask(df['DeviceType']=='tablet',2)
    df = df.drop(columns='DeviceType')
    # Browser
    df['browser_enc2'] = 7
    df['browser_enc2'] = df['browser_enc2'].mask(df['browser_enc']=='chrome',0)
    df['browser_enc2'] = df['browser_enc2'].mask(df['browser_enc']=='safari',1)
    df['browser_enc2'] = df['browser_enc2'].mask(df['browser_enc']=='firefox',2)
    df['browser_enc2'] = df['browser_enc2'].mask(df['browser_enc']=='ie',3)
    df['browser_enc2'] = df['browser_enc2'].mask(df['browser_enc']=='android',4)
    df['browser_enc2'] = df['browser_enc2'].mask(df['browser_enc']=='edge',5)
    df['browser_enc2'] = df['browser_enc2'].mask(df['browser_enc']=='opera',6)
    df = df.drop(columns='browser_enc')
    # Device info
    df['device_info_v4_enc'] = 17
    df['device_info_v4_enc'] = df['device_info_v4_enc'].mask(df['device_info_v4']=='ios',0)
    df['device_info_v4_enc'] = df['device_info_v4_enc'].mask(df['device_info_v4']=='windows',1)
    df['device_info_v4_enc'] = df['device_info_v4_enc'].mask(df['device_info_v4']=='samsung',2)
    df['device_info_v4_enc'] = df['device_info_v4_enc'].mask(df['device_info_v4']=='hisense',3)
    df['device_info_v4_enc'] = df['device_info_v4_enc'].mask(df['device_info_v4']=='moto',4)
    df['device_info_v4_enc'] = df['device_info_v4_enc'].mask(df['device_info_v4']=='pixel',5)
    df['device_info_v4_enc'] = df['device_info_v4_enc'].mask(df['device_info_v4']=='lg',6)
    df['device_info_v4_enc'] = df['device_info_v4_enc'].mask(df['device_info_v4']=='blade',7)
    df['device_info_v4_enc'] = df['device_info_v4_enc'].mask(df['device_info_v4']=='huawei',8)
    df['device_info_v4_enc'] = df['device_info_v4_enc'].mask(df['device_info_v4']=='oneplus',9)
    df['device_info_v4_enc'] = df['device_info_v4_enc'].mask(df['device_info_v4']=='alcatel',10)
    df['device_info_v4_enc'] = df['device_info_v4_enc'].mask(df['device_info_v4']=='redmi',11)
    df['device_info_v4_enc'] = df['device_info_v4_enc'].mask(df['device_info_v4']=='lenovo',12)
    df['device_info_v4_enc'] = df['device_info_v4_enc'].mask(df['device_info_v4']=='asus',13)
    df['device_info_v4_enc'] = df['device_info_v4_enc'].mask(df['device_info_v4']=='linux',14)
    df['device_info_v4_enc'] = df['device_info_v4_enc'].mask(df['device_info_v4']=='android',15)
    df['device_info_v4_enc'] = df['device_info_v4_enc'].mask(df['device_info_v4']=='zte',16)
    df = df.drop(columns='device_info_v4')
    # Index
    df = df.set_index('customer_id')
    return df

In [14]:
df = clustering_encoding(df)

In [15]:
df.sample(4)

,TransactionAmt,max_c,max_d,product_enc,card4_enc,card6_enc,DeviceType_enc,browser_enc2,device_info_v4_enc
customer_id,,,,,,,,,
2570,0.032845,0.000000,0.040281,3,0,1,0,0,2
2663,0.049520,0.000000,0.000000,1,0,1,0,1,0
100,0.026175,0.000534,0.012297,2,1,1,0,0,0
1741,0.032845,0.000000,0.000000,3,0,0,0,1,0


# 3.0 Personas generation

In [16]:
def clustering_main(df,version,max_cluster,choose_n_cluster):
    """
    
    """
    if version == 'training':
        sil=[]
        dft = df.copy()
        for num_clusters in list(range(2,max_cluster)):
            kproto = KPrototypes(n_clusters=num_clusters, verbose=0,random_state = 301,n_init=5)
            cluster_labels = kproto.fit_predict(dft.values, categorical=[3,4,5,6,7,8])
            dft['cluster_labels'] = cluster_labels
            dist = gower.gower_matrix(dft.drop(columns=['cluster_labels']),cat_features = [False, False ,False, True, True,True,True,True,True])
            sil.append(silhouette_score(dist,labels=cluster_labels, metric='precomputed'))
            print('For cluster number: ',num_clusters,' the score is: ', sil[-1])
    elif version == 'choosen':
        kproto = KPrototypes(n_clusters=choose_n_cluster, verbose=0,random_state = 301)
        cluster_labels = kproto.fit_predict(df.values, categorical=[3,4,5,6,7,8])
        centroid = []
        for j,i in enumerate(kproto.cluster_centroids_):
            centroid.append(pd.DataFrame(i,index=['TransactionAmt','max_c', 'max_d','product_enc', 'card4_enc', 'card6_enc',
               'DeviceType_enc', 'browser_enc2', 'device_info_v4_enc'],columns=[j]))
        centroid = pd.concat(centroid,axis=1)
        centroid.style.background_gradient(cmap='brg',axis=1)
        df['cluster_labels'] = cluster_labels
        print(df['cluster_labels'].value_counts())
        return df, centroid, kproto

In [24]:
clustering_main(df,'training',5,None)

For cluster number:  2  the score is:  0.26329783
For cluster number:  3  the score is:  0.2508759
For cluster number:  4  the score is:  0.21009842


In [17]:
df, centroid, model = clustering_main(df,'choosen',None,4)

0    2969
2    2109
3    1918
1    1031
Name: cluster_labels, dtype: int64


In [18]:
centroid.style.background_gradient(cmap='brg',axis=1)

,0,1,2,3
TransactionAmt,0.060090,0.080473,0.027653,0.066001
max_c,0.006742,0.005659,0.012320,0.000806
max_d,0.048834,0.325682,0.059076,0.015510
product_enc,0.000000,3.000000,0.000000,1.000000
card4_enc,0.000000,0.000000,0.000000,0.000000
card6_enc,1.000000,0.000000,0.000000,1.000000
DeviceType_enc,1.000000,1.000000,0.000000,0.000000
browser_enc2,0.000000,0.000000,0.000000,1.000000
device_info_v4_enc,1.000000,1.000000,17.000000,0.000000


# 4.0 Prediction

In [19]:
def clustering_prediction(df,model):
    """
    
    """
    df['cluster_labels_pred'] = model.predict(X=df[['TransactionAmt', 'max_c', 'max_d',
                                                    'product_enc', 'card4_enc', 'card6_enc', 'DeviceType_enc','browser_enc2',
                                                    'device_info_v4_enc']],categorical=[3,4,5,6,7,8])
    return df

In [20]:
df = clustering_prediction(df,model)

In [21]:
df.sample(4)

,TransactionAmt,max_c,max_d,product_enc,card4_enc,card6_enc,DeviceType_enc,browser_enc2,device_info_v4_enc,cluster_labels,cluster_labels_pred
customer_id,,,,,,,,,,,
119,0.008803,0.000000,0.252373,0,0,1,1,0,1,0,0
235,0.019505,0.055842,0.170981,2,1,0,1,0,1,1,1
33020,0.007708,0.000000,0.000000,0,1,1,0,1,17,3,3
5360,0.066195,0.000000,0.130334,3,0,0,1,0,1,1,1


# 5.0  Output

In [22]:
with open('../../data_lake/output/clustering_model_v1.pkl','wb') as file:
    pickle.dump(model, file)

In [23]:
with open('../../data_lake/output/clustering_imputation_num.pkl','wb') as file:
    pickle.dump(imp_mean, file)

In [24]:
with open('../../data_lake/output/clustering_imputation_cat.pkl','wb') as file:
    pickle.dump(imp_mean, file)

In [25]:
with open('../../data_lake/output/clustering_scaler_num.pkl','wb') as file:
    pickle.dump(scaler, file)